In [5]:
DEBUG = False

# Word file assists with the randomness of the prompt
with open("words", 'r') as file:
        lines_array = [line.strip() for line in file.readlines()]
        if(DEBUG):
                print(f"{len(lines_array)} words in list")

In [6]:
from ollama import Client
from ollama import generate
import random

def generate_prompt():
    word1 = lines_array[random.randint(0, len(lines_array))]
    word2 = lines_array[random.randint(0, len(lines_array))]
    prompt = f"The theme is '{word1} {word2}'."
    if(DEBUG):
        print(f"{prompt}")
    # print(f"{word1}:{word2}")
    return prompt

def generate_response(client, model):
    prompt = generate_prompt()
    seed = int(random.random()*1000)
    system_prompt = "Write a single sentence with between 8 and 10 words. Only output the sentence generated without preamble. The goal is to use all the letters of the alphabet."
    options = {
        "seed": seed, 
        "temperature": seed/100.0,
        "num_predict": 400

    }
    response = client.generate(
        model = model,
        prompt = f"{system_prompt} {prompt}",
        # system = system_prompt,
        options=options,
        stream = False
    )
    reply = response['response']
    if(DEBUG):
        print(f"r:{reply}")
    return reply


In [7]:
class Sentence:
    boring_words = ["quick", "brown", "fox", "jumps", "dog", "lazy"]
    def __init__(self, text, client):
        """
        Args:
            text (str): The text of the sentence.  Must be a string.
        
        Raises:
            TypeError: if input is not a string
        """
        if not isinstance(text, str):
            raise TypeError("Input must be a string.")

        if len(text) > 80:
            if(DEBUG):
                print(f"long text rejected: {text}")
            raise TypeError("No paragraphs.")
        
        self._text = text.encode("ascii", errors="ignore").decode()  # Use a protected attribute for encapsulation
        self._words = len(text.split(" "))
        self._missing = self.find_missing_letters()
        self._duplicates = self.find_duplicate_letters()
        self._duplength = len(self._duplicates)
        self._dupkeys = "".join(self._duplicates.keys())
        self._client = client

    def get_text(self):
        """
        Returns the text of the sentence.

        Returns:
            str: The sentence text.
        """
        return self._text

    def get_client(self):
        return self._client
    
    def get_missing(self):
        return self._missing
    
    def get_missing_count(self):
        return len(self.get_missing())
      
    def get_duplicates(self):
        return self._duplicates
    
    def get_duplength(self):
        return self._duplength
    
    def get_dupkeys(self):
        return self._dupkeys
    
    def get_score(self):
        missing_score = (26 - self.get_missing_count())**2 if self.get_missing_count() < 3 else 1
        duplicate_score = self.get_duplength()
        for boring_word in self.boring_words:
            if(boring_word in self.get_text().lower()):
                if(DEBUG):
                    print(f"Lowering score for {boring_word}")
                missing_score = missing_score - 100
        return missing_score * duplicate_score

    
    def find_missing_letters(self):
        """
        Checks a string for the presence of all lowercase letters and identifies missing ones.

        Returns:
            A string containing the missing letters, or an empty string if all letters are present.
            Returns None if the input is not a string.
        """

        alphabet = "abcdefghijklmnopqrstuvwxyz"
        missing_letters = ""
        
        # Convert the input string to lowercase for case-insensitive comparison
        input_string = self._text.lower()

        for letter in alphabet:
            if letter not in input_string:
                missing_letters += letter

        return missing_letters

    def find_duplicate_letters(self):
        """
        Identifies duplicate letters in a string and counts their occurrences.

        Returns:
            A dictionary where keys are the duplicate letters and values are their counts.
            Returns an empty dictionary if there are no duplicates.
            Returns None if the input is not a string.
        """

        letter_counts = {}
        duplicate_counts = {}

        input_string = self._text.lower()  # Case-insensitive counting

        for letter in input_string:
            if 'a' <= letter <= 'z':  # Only consider letters
                if letter in letter_counts:
                    letter_counts[letter] += 1
                else:
                    letter_counts[letter] = 1

        # Identify duplicates and create a new dictionary
        for letter, count in letter_counts.items():
            if count > 1:
                duplicate_counts[letter] = count

        return duplicate_counts

class Winners:
    def __init__(self):
        self._sentences = []

    def add(self, sentence):
        self._sentences.append(sentence)
        if(DEBUG):
            print(f"There are {len(self._sentences)} sentences in the list.")

    def get_missing_winners(self, _count):
        sorted_winners = sorted(self._sentences, key=lambda item: item.get_missing_count())
        return sorted_winners[:_count]
    
    def get_duplicate_winners(self, _count):
        sorted_winners = sorted(self._sentences, key=lambda item: item.get_score())
        return sorted_winners[len(sorted_winners)-_count:]

    def length(self):
        return len(self._sentences)

# Test cases. Add to this list when you find good terms.
the_sentences = [
    "The quick brown fox jumps over the lazy dog.",
    "Golden sunlight danced upon the rippling, turquoise ocean waves.",
    "Old photographs revealed stories of laughter, love, and bygone days.",
    "The majestic mountains stood silently guarding the peaceful valley below.",
    "She expertly crafted intricate jewelry using shimmering silver and gold. The majestic mountains stood silently guarding the peaceful valley below.",
    "Dark storm clouds gathered quickly, threatening a powerful summer rain.",
    "Street food tasted wonderfully fresh.",
    "Remote islands felt truly peaceful.",
    "The axel, quickly! Jumping fences with brave, zealous strides.",
    "The teen quickly jammed, exploring vibrant chords with joyful zest."
]

winners = Winners()
for sentence in the_sentences:
    try:
        winners.add(Sentence(sentence, 0))
        
    except Exception as e:
        print(e)


missing_winners = winners.get_missing_winners(5)
for winner in missing_winners:
    print(f"missing {winner.get_missing_count()} {winner.get_text()} '{winner.get_missing()}' duplicates: {winner.get_duplength()} '{winner.get_dupkeys()}' client: {winner.get_client()}")

print("****************************")
duplicate_winners = winners.get_duplicate_winners(5)
for winner in duplicate_winners:
    print(f"duplicates: {winner.get_duplength()} {winner.get_text()} '{winner.get_dupkeys()}' missing {winner.get_missing_count()} '{winner.get_missing()}' client: {winner.get_client()}")



No paragraphs.
missing 0 The quick brown fox jumps over the lazy dog. '' duplicates: 6 'theuro' client: 0
missing 0 The axel, quickly! Jumping fences with brave, zealous strides. '' duplicates: 11 'thealuicnsr' client: 0
missing 0 The teen quickly jammed, exploring vibrant chords with joyful zest. '' duplicates: 16 'thenuiclyjamdors' client: 0
missing 4 The majestic mountains stood silently guarding the peaceful valley below. 'kqxz' duplicates: 15 'themasicoundlyg' client: 0
missing 5 Dark storm clouds gathered quickly, threatening a powerful summer rain. 'bjvxz' duplicates: 16 'darkstomclughein' client: 0
****************************
duplicates: 15 Golden sunlight danced upon the rippling, turquoise ocean waves. 'goldensuihtacpr' missing 8 'bfjkmxyz' client: 0
duplicates: 15 The majestic mountains stood silently guarding the peaceful valley below. 'themasicoundlyg' missing 4 'kqxz' client: 0
duplicates: 6 The quick brown fox jumps over the lazy dog. 'theuro' missing 0 '' client: 0
dup

In [9]:
import threading
import time

timeout = 60*30 #7*60*60 #30*30 #1*60*60 #2*60*60 #7*60*60
winning_sentences = Winners()
running = True
previous_missing = [] # Keep track of high winners so we can display it when it changes.
DEBUG = False

def worker(client_id, url, model):
    """
     a client processing a task.
    
    Args:
        client_id: Unique identifier for the client.
        url: The URL associated with the client's task.
        model: the model to use for this client.
    """
    client = Client(host=url)
    i = 0
    while True:
        global running
        global winning_sentences
        global previous_missing
        i = i + 1
        if(running != True):
           # Exiting
           print(f"Quitting client {client_id} url {url} model {model} i {i} running: {running}")
           return
        try:
            if(DEBUG):
                print(f"client {client_id} url {url} model {model} i {i} running: {running}")

            sentence = Sentence(generate_response(client, model).strip().replace("\n", " "), client_id)
            if(DEBUG):
                print(sentence.get_text())

            winning_sentences.add(sentence)

            missing_winners = winning_sentences.get_duplicate_winners(5)
            if(missing_winners != previous_missing):
                previous_missing = missing_winners
                for winner in missing_winners:
                    print(f"missing {winner.get_missing_count()} {winner.get_text()} '{winner.get_missing()}' duplicates: {winner.get_duplength()} '{winner.get_dupkeys()}' client {winner.get_client()} score: {winner.get_score()}")
                print("****************************\n")
            # 3 Seconds of cool down for the LLM. This actually saves time in the forever loop.
            time.sleep(3)

        except Exception as e:
            dummy = 1 # Ignore all exceptions. Set dummy so linter doesn't complain.
            if(DEBUG):
                print(f"client {client_id} url {url} {e}")
        

def main():
    """
    Creates and starts multiple client threads.
    """

    clients = [
        {"id": 1, "url": "http://192.168.137.118:11434", "model": "gemma3:12b"},
        {"id": 2, "url": "http://192.168.137.117:11434", "model": "gemma3:4b"},
        {"id": 3, "url": "http://192.168.137.119:11434", "model": "gemma3:4b"},
    ]

    global running
    running = True
    threads = []
    for client in clients:
        thread = threading.Thread(target=worker, args=(client["id"], client["url"], client["model"]))
        threads.append(thread)
        thread.start()

    try:
        time.sleep(timeout)
        # Time's up. Tell threads to wrap it up. There may be a significant delay because the client might still be processing.
        running = False
        print(f"Calling it quits after {timeout} seconds.")
        
        # Wait for all threads to finish
        for thread in threads:
            thread.join(timeout=30)

        print("\n\nAll threads have stopped.")
        missing_winners = winning_sentences.get_missing_winners(50)
        
        # Word file assists with the randomness of the prompt
        with open("winners", 'a') as file:
            for winner in missing_winners:
                message = f"missing {winner.get_missing_count()} {winner.get_text()} '{winner.get_missing()}' duplicates: {winner.get_duplength()} '{winner.get_dupkeys()}' client {winner.get_client()} score: {winner.get_score()}"
                print(message)
                print(message, file=file)

            print("-----------------------------")
            duplicate_winners = winning_sentences.get_duplicate_winners(50)
            for winner in duplicate_winners:
                message = f"duplicates: {winner.get_duplength()} {winner.get_text()} '{winner.get_dupkeys()}' missing {winner.get_missing_count()} '{winner.get_missing()}' client {winner.get_client()} score: {winner.get_score()}"
                print(message)
                print(message, file=file)

            file.close()
            print("============================\n\n")

    
       
    except KeyboardInterrupt:
        print("Stopping threads due to KeyboardInterrupt...")
        running = False
        for thread in threads:
            thread.join()
        print("All threads stopped after KeyboardInterrupt")

if __name__ == "__main__":
    main()

missing 8 Whereas glad xylophones zigzayed wildly each zooying zebra. 'fjkmqtuv' duplicates: 14 'wherasgldyonzi' client 3 score: 14
****************************

missing 8 Whereas glad xylophones zigzayed wildly each zooying zebra. 'fjkmqtuv' duplicates: 14 'wherasgldyonzi' client 3 score: 14
missing 2 Quiz brought every joyful keyword, framing historic values. 'px' duplicates: 14 'uiroghtevyflas' client 1 score: 8064
****************************

missing 3 Observe exotic xylographs, just beyond my fragile yet potent, unique orbit! 'kwz' duplicates: 15 'obserxtiylgapun' client 2 score: 15
missing 2 Quiz brought every joyful keyword, framing historic values. 'px' duplicates: 14 'uiroghtevyflas' client 1 score: 8064
****************************

missing 2 Quiz brought every joyful keyword, framing historic values. 'px' duplicates: 14 'uiroghtevyflas' client 1 score: 8064
****************************

missing 8 Whereas glad xylophones zigzayed wildly each zooying zebra. 'fjkmqtuv' duplica